# Эмбеддинги

## Задание

Возьмите данные соревнования по определению перефразирования - http://paraphraser.ru/download/get?file_id=1
Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec, Fastext, Adagram.Для обучения этих моделей можете воспользоваться корпусом новостных текстов, с которым мы работали на 4 и 5 семинарах. А можете использовать любой другой корпус (сами тексты соревнования использовать не надо).

У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

Постройте обучающую выборку из этих близостей. Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).   

Предложите любой способ улучшить эту метрику (и реализуйте его!).

Оценивание - если вы сделали всё вышеперечисленное - 10 баллов. Каждая ошибка - минус 0.5 балла. 

В выполнении задания вам могут помочь тетрадки семинаров 4 и 5 (они лежат в папке notebooks гитхаба курса).П
Выложите код к себе на гитхаб и вставьте ссылку в поле ниже (в тетрадке сохраните показатели метрик).одготовка


## Подготовка эмбеддингов

In [1]:
import pandas as pd
from lxml import html
import numpy as np
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymystem3 import Mystem

mystem = Mystem()

In [2]:
def mystem_tokenize(text):
    return [w for w in mystem.lemmatize(text) if w not in ' \n']

In [3]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

### Считываем данные для обучения

Для обучения эмбеддингов я взяла [корпус Ленты.ру](https://github.com/yutkin/Lenta.Ru-News-Dataset).

In [4]:
news_df = pd.read_csv('data/news_lenta.csv')
news_df.head()

,tags,text,title,topic,url
0,Бокс и ММА,"Украинский боксер Александр Усик рассказал, чт...",Усик поделился планами на будущее после победы...,Спорт,https://lenta.ru/news/2018/07/22/usik_plany/
1,Политика,Израиль эвакуировал из Сирии активистов «Белых...,Израиль вывез из Сирии членов «Белых касок» по...,Мир,https://lenta.ru/news/2018/07/22/evacuation/
2,Футбол,Лондонский «Челси» отказался продать мадридско...,«Челси» отказался продать футболиста «Реалу» з...,Спорт,https://lenta.ru/news/2018/07/22/hazard_real/
3,Бокс и ММА,Вице-президент Федерации профессионального бок...,Слова Порошенко о победе Усика сочли идиотизмом,Спорт,https://lenta.ru/news/2018/07/22/otvet/
4,Музыка,Журнал Billboard опубликовал рейтинг самых выс...,Названы самые богатые музыканты,Культура,https://lenta.ru/news/2018/07/22/moneyrich/


In [5]:
news_df.dropna(inplace=True)

Мешок слов на `CountVectorizer`. Так как корпус большой и обрабатывается слишком долго, возьмём только первые 5000 статей.

In [6]:
cv = CountVectorizer(min_df=2, max_features=1000)
bow = cv.fit_transform(normalize(text) for text in news_df['text'][:5000])
bow.shape

(5000, 1000)

### Снижение размерности с SVD

In [7]:
svd = TruncatedSVD(300)
svd.fit(bow)

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=None, tol=0.0)

In [8]:
id2word = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id = {w:i for i,w in id2word.items()}

id2vec_svd = svd.components_.T

Поиграемся с моделью:

In [9]:
def most_similar(word, id2vec):
    return [id2word[i] for i in cosine_distances(id2vec[word2id[word]].reshape(1, -1), id2vec).argsort()[0][:10]]

most_similar('спорт', id2vec_svd)

['спорт',
 'спортивный',
 'спортсмен',
 'стадион',
 'открытый',
 'футбольный',
 'проводиться',
 'выступление',
 'четверг',
 'футболист']

In [10]:
sp = id2vec_svd[word2id['спорт']]
spm = id2vec_svd[word2id['спортсмен']]
cosine_similarity([sp], [spm])[0][0]

0.3104514754061739

### Снижение размерности с NMF

In [11]:
nmf = NMF(100)
nmf.fit(bow)
id2vec_nmf = nmf.components_.T

In [12]:
most_similar('спорт', id2vec_nmf)

['спорт',
 'футбол',
 'чемпионат',
 'спортивный',
 'мундиаля',
 'турнир',
 'мир',
 'фифа',
 'федерация',
 'должность']

### Word2vec

In [13]:
w2v = gensim.models.Word2Vec([normalize(text).split() for text in news_df['text']], size=50, sg=1)

KeyboardInterrupt: 

## Классификация

### Считываем данные для классификации

In [ ]:
! tail -n 20 data/paraphrases.xml

In [14]:
with open('data/paraphrases.xml') as f:
    data = f.read()

In [15]:
from xml.etree import ElementTree

In [16]:
root = ElementTree.XML(data)

In [17]:
par_data = {'text1': [], 'text2': [], 'class': []}
for par in root[1]:
    par_data['text1'].append(par[3].text)
    par_data['text2'].append(par[4].text)
    par_data['class'].append(int(par[6].text))

In [18]:
parphrase_df = pd.DataFrame(par_data)
parphrase_df.head(10)

,class,text1,text2
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...


### Строим обучающий датасет

In [21]:
id2vec = id2vec_nmf
def get_sent_similarity(line):
    """
    сделаем максимально тупо: получаем эмбеддинг предложния усреднением эмбеддингов слов
    """
    sent1 = line['text1']
    sent2 = line['text2']
    try:
        vec1 = [id2vec[word2id[word]] for word in mystem_tokenize(sent1) if word in word2id]
        vec1 = sum(vec1) / len(vec1)
        vec2 = [id2vec[word2id[word]] for word in mystem_tokenize(sent2) if word in word2id]
        vec2 = sum(vec2) / len(vec2)
        return cosine_similarity([vec1], [vec2])[0][0]
    except ZeroDivisionError:
        return 0

In [20]:
parphrase_df['svd_sim'] = parphrase_df.apply(get_sent_similarity, axis=1)

In [22]:
parphrase_df['nmf_sim'] = parphrase_df.apply(get_sent_similarity, axis=1)

In [37]:
parphrase_df['metrics'] = parphrase_df.apply(lambda x: [x['svd_sim'], x['nmf_sim']], axis=1)

### Учим логрегрессию

In [44]:
x_train, x_test, y_train, y_test = train_test_split(parphrase_df['metrics'], parphrase_df['class'])

In [48]:
x_train = [np.array(x) for x in x_train]
x_test = [np.array(x) for x in x_test]

In [49]:
clf = LogisticRegression(random_state=42)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

         -1       0.51      0.56      0.54       655
          0       0.46      0.68      0.55       744
          1       0.00      0.00      0.00       408

avg / total       0.38      0.48      0.42      1807



/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
